In [ ]:
import psycopg2
import hidden_example
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# Load the secrets
secrets = hidden_example.secrets()


conn = psycopg2.connect(host=secrets['host'],
        port=secrets['port'],
        database=secrets['database'], 
        user=secrets['user'], 
        password=secrets['pass'], 
        connect_timeout=3)
try:
    db = conn.get_dsn_parameters()['dbname']
    print(f"Successfully connected to the database: {db}")
except Exception as e:
    print(e)
    
# defining widgets
cat_widget = widgets.Dropdown(
    options=[('Income', 1), ('Household expenses', 2), ('Fruit', 3), ('Veggies', 4), ('Protein', 5), ('Groceries', 6), ('Relax', 7)],
    placeholder='Select a category',
    description='Category:',
)

date_widget = widgets.DatePicker(
    description='Transaction date:',
    placeholder='Select a date',
    disabled=False
)

detail_widget = widgets.Text(
    placeholder='Insert transaction detail',
    description='Transaction detail:',
    disabled=False
)

amount_widget = widgets.FloatText(
    description='Total amount:',
    disabled=False
)

add_row_button = widgets.Button(description="Add Row")
delete_row_button = widgets.Button(description="Delete Row")
submit_button = widgets.Button(description='Submit', button_style='success')

output_buttons = widgets.Output()

# Global list to store rows
pending_rows = []

def on_add_row(button):
    # Get and validate the input
    cat_id = cat_widget.value
    date = date_widget.value.strftime('%Y-%m-%d') if date_widget.value else None
    detail = detail_widget.value
    amount = amount_widget.value
    with output_buttons:
        output_buttons.clear_output()
        if not (cat_id and date and detail and amount):
            print('Please fill all the fields')
            return

        row = [cat_id, date, detail, amount]
        pending_rows.append(row)
        rows_df = pd.DataFrame(pending_rows, columns=['cat_id', 'tran_date', 'detail', 'total_amount'])
        print(f"Row added:")
        display(rows_df)  # Display the current pending rows

def on_delete_row(button):
    with output_buttons:
        output_buttons.clear_output()
        if not pending_rows:
            print("No rows to delete")
            return
        pending_rows.pop()  # Remove the last row
        if not pending_rows:
            print("All rows deleted, no pending rows left")
            return
        rows_df = pd.DataFrame(pending_rows, columns=['cat_id', 'tran_date', 'detail', 'total_amount'])
        print(f"Row deleted, remaining rows:")
        display(rows_df)
       
def on_submit_all(button):
    with output_buttons:
        output_buttons.clear_output()
        if not pending_rows:
            print("No rows to insert")
            return
        sql = """INSERT INTO transactions (cat_id, tran_date, detail, total_amount) VALUES (%s, %s, %s, %s)"""
        try:
            with conn.cursor() as cur:
                cur.executemany(sql, pending_rows)
            conn.commit()
            print(f"{len(pending_rows)} transaction(s) inserted successfully")
            rows_df = pd.DataFrame(pending_rows, columns=['cat_id', 'tran_date', 'detail', 'total_amount'])
            display(rows_df)
            pending_rows.clear()  # Clear the list after insertion
        except Exception as e:
            conn.rollback()
            print(f'Error inserting transactions: {e}')



add_row_button.on_click(on_add_row)
delete_row_button.on_click(on_delete_row)
submit_button.on_click(on_submit_all)

display(cat_widget, date_widget, detail_widget, amount_widget, add_row_button, delete_row_button, submit_button, output_buttons)